In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

PROJECT_JSON = kraft.read_json("../project.json")

PATH = make_path_dict(PROJECT_JSON)

In [ ]:
if PROJECT_JSON["signal_type"] != "raw":

    raise

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = PROJECT_JSON["sample_alias"]

In [ ]:
xaxis = {"title": "Rank"}

yaxis = {"title": "Raw Signal Standard Deviation"}

In [ ]:
if PROJECT_JSON["select_feature_automatically"]:

    features = kraft.select_series_indices(
        feature_x_sample_processed.std(axis=1),
        ">",
        standard_deviation=PROJECT_JSON["select_feature_standard_deviation"],
        title={"text": feature_x_sample_processed.index.name},
        xaxis=xaxis,
        yaxis=yaxis,
    )

else:

    features = feature_x_sample_processed.index

f"Selected {features.size} {PROJECT_JSON['feature_alias']}."

In [ ]:
if PROJECT_JSON["select_sample_automatically"]:

    samples = kraft.select_series_indices(
        feature_x_sample_processed.std(),
        ">",
        standard_deviation=PROJECT_JSON["select_sample_standard_deviation"],
        title={"text": feature_x_sample_processed.columns.name},
        xaxis=xaxis,
        yaxis=yaxis,
    )

else:

    samples = feature_x_sample_processed.columns

f"Selected {samples.size} {PROJECT_JSON['sample_alias']}."

In [ ]:
raw_signal = feature_x_sample_processed.loc[features, samples]

In [ ]:
if PROJECT_JSON["signal_normalization_method"] is not None:

    raw_signal = kraft.normalize_series_or_dataframe(
        raw_signal,
        PROJECT_JSON["signal_normalization_axis"],
        PROJECT_JSON["signal_normalization_method"],
    )

In [ ]:
raw_signal.to_csv(PATH["feature_x_sample.signal.tsv"], sep="\t")

kraft.summarize_feature_x_sample(
    raw_signal,
    feature_x_sample_alias=PROJECT_JSON["feature_x_sample_alias"],
    feature_x_sample_value_name="Signal",
    plot_heat_map_max_size=PROJECT_JSON["plot_heat_map_max_size"],
    plot_histogram_max_size=PROJECT_JSON["plot_histogram_max_size"],
    plot_rug_max_size=PROJECT_JSON["plot_rug_max_size"],
)